In [9]:
pip install pywin32

Looking in indexes: https://pypi.org/simple/
  Using cached pywin32-307-cp312-cp312-win_amd64.whl.metadata (8.3 kB)
Using cached pywin32-307-cp312-cp312-win_amd64.whl (6.5 MB)
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\User\\anaconda3\\envs\\Python_v1\\Lib\\site-packages\\pywin32_system32\\pythoncom312.dll'
Consider using the `--user` option or check the permissions.



# 변경 XLSX > XL

In [10]:
import win32com.client as win32
import os

# 원본 파일 경로와 저장할 파일 경로 설정
original_file = r'd:\시설\유지\24.9월유지가입자.xlsx'  # 보안이 걸린 원본 파일
new_file = r'd:\시설\유지\24.9월유지가입자_보안해제2.xl'  # 최종적으로 저장할 파일 (.xl 확장자)
try:
    # 엑셀 애플리케이션 실행
    excel = win32.Dispatch("Excel.Application")
    excel.Visible = False  # 엑셀 창을 보이지 않도록 설정 (필요 시 True로 변경)
    excel.DisplayAlerts = False  # 엑셀 경고 메시지 비활성화

    # 빈 엑셀 파일 생성
    workbook = excel.Workbooks.Add()  # 빈 엑셀 파일 생성

    # 보호된 파일 열기 (보호된 모드 해제)
    protected_workbook = excel.Workbooks.Open(Filename=original_file, ReadOnly=False)
    # 데이터 복사: 모든 시트를 새로운 워크북으로 복사
    for sheet in protected_workbook.Sheets:
        sheet.Copy(Before=workbook.Sheets(1))  # 빈 엑셀 파일에 시트 복사
    # 보호된 파일 닫기
    protected_workbook.Close(SaveChanges=False)
    # 새로운 파일로 저장 (원하는 확장자로 저장)
    workbook.SaveAs(Filename=new_file, FileFormat=51)  # 51은 .xlsx 확장자, 원하는 경우 .xl 확장자로 설정
    print(f"파일이 성공적으로 저장되었습니다: {new_file}")
    # 엑셀 애플리케이션 종료
    workbook.Close(SaveChanges=False)
    excel.Quit()
except Exception as e:
    print(f"파일 처리 중 오류 발생: {e}")
finally:

    # 엑셀 애플리케이션 강제 종료 (오류 발생 시 메모리 해제)
    excel.Quit()

파일이 성공적으로 저장되었습니다: d:\시설\유지\24.9월유지가입자_보안해제2.xl


# 대량파일 3개분리하고 ZIP파일 열면 하나로 저장 xlsx

In [ ]:
import os
import zipfile
from openpyxl import load_workbook, Workbook

# 원본 파일 경로 및 변경된 파일 경로 설정 (.xlsx 확장자로 변경)
original_file = r'd:\시설\유지\24.9월유지가입자_보안해제2.xl'
corrected_file = r'd:\시설\유지\24.9월유지가입자_보안해제2.xlsx'

# 원본 파일을 .xlsx 형식으로 변경
try:
    if os.path.exists(original_file):
        os.rename(original_file, corrected_file)
        print(f"파일 확장자가 성공적으로 변경되었습니다: {original_file} -> {corrected_file}")
    else:
        print(f"지정된 파일이 존재하지 않습니다: {original_file}")
except Exception as e:
    print(f"파일 확장자 변경 중 오류 발생: {e}")

# 변경된 파일 경로로 파일을 읽음
split_dir = r'd:\시설\유지\분할파일'
zip_file_path = r'd:\시설\유지\유지가입자분할.zip'
split_count = 3

if not os.path.exists(split_dir):
    os.makedirs(split_dir)

# 올바른 확장자로 파일 읽기
wb = load_workbook(filename=corrected_file)
ws = wb.active

# 데이터 분할하기
rows = list(ws.iter_rows(values_only=True))
total_rows = len(rows)
rows_per_split = total_rows // split_count

# 각 분할 파일을 생성
for i in range(split_count):
    split_wb = Workbook()
    split_ws = split_wb.active
   
    start_row = i * rows_per_split
    end_row = (i + 1) * rows_per_split if i != split_count - 1 else total_rows
   
    for row in rows[start_row:end_row]:
        split_ws.append(row)
   
    split_filename = os.path.join(split_dir, f'유지가입자_part_{i + 1}.xlsx')
    split_wb.save(split_filename)
    print(f"파일 생성 완료: {split_filename}")

# ZIP 파일로 압축
with zipfile.ZipFile(zip_file_path, 'w') as zipf:
    for root, _, files in os.walk(split_dir):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, arcname=file)
            print(f"파일 압축 중: {file_path}")

print(f"분할 파일이 성공적으로 ZIP 압축되었습니다: {zip_file_path}")

파일 확장자가 성공적으로 변경되었습니다: d:\시설\유지\24.9월유지가입자_보안해제2.xl -> d:\시설\유지\24.9월유지가입자_보안해제2.xlsx


In [2]:
pip install pandas tqdm openpyxl

Looking in indexes: https://pypi.org/simple/
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import pandas as pd
import zipfile
from tqdm import tqdm

# 원본 파일 경로 및 변경된 파일 경로 설정
original_file = r'd:\시설\유지\24.9월유지가입자_보안해제2.xlsx'

# 분할 파일 저장 경로 및 ZIP 파일 경로 설정
split_dir = r'd:\시설\유지\분할파일'
zip_file_path = r'd:\시설\유지\유지가입자분할.zip'

# 분할할 파일 개수
split_count = 3

# 저장할 폴더가 없으면 생성
if not os.path.exists(split_dir):
    os.makedirs(split_dir)

# Excel 파일을 pandas 데이터프레임으로 읽기 (더 빠른 처리)
print("원본 파일 읽는 중...")
df = pd.read_excel(original_file, engine='openpyxl')
print(f"파일 읽기 완료: {original_file} (총 {len(df)} 행)")

# 분할 데이터 개수 계산
rows_per_split = len(df) // split_count

# 분할 파일 생성 및 저장
print("파일 분할 작업 중...")
for i in tqdm(range(split_count), desc="분할 진행", unit="파일"):
    start_row = i * rows_per_split
    end_row = (i + 1) * rows_per_split if i != split_count - 1 else len(df)
   
    # 데이터프레임을 해당 범위만큼 슬라이싱하여 새로운 파일로 저장
    split_df = df.iloc[start_row:end_row]
   
    # 분할 파일 저장 경로
    split_filename = os.path.join(split_dir, f'유지가입자_part_{i + 1}.xlsx')
    split_df.to_excel(split_filename, index=False)
    print(f"파일 생성 완료: {split_filename} (행: {start_row + 1} ~ {end_row})")

# 분할된 파일을 ZIP 파일로 압축
print("파일 압축 작업 중...")
with zipfile.ZipFile(zip_file_path, 'w') as zipf:
    for root, _, files in os.walk(split_dir):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, arcname=file)  # 각 파일을 ZIP에 추가
            print(f"파일 압축 중: {file_path}")

print(f"분할 파일이 성공적으로 ZIP 압축되었습니다: {zip_file_path}")

원본 파일 읽는 중...
파일 읽기 완료: d:\시설\유지\24.9월유지가입자_보안해제2.xlsx (총 391963 행)
파일 분할 작업 중...


분할 진행:   0%|                                                                               | 0/3 [13:38<?, ?파일/s]


KeyboardInterrupt: 

## 파일의 크기가 큰 경우, 분할 및 저장 속도를 더 빠르게 하기 위해 다음과 같은 추가 최적화 방법을 적용할 수 있습니다. 대용량 파일은 파일을 읽고 쓰는 데 시간이 많이 걸리기 때문에, I/O (입출력) 작업을 줄이는 것이 중요합니다. 또한, 데이터 처리 속도를 높이기 위해 몇 가지 추가적인 방안을 적용할 수 있습니다.

### 최적화 방법:
1. **`pandas`의 `chunksize` 사용**:
   - 파일을 한 번에 읽는 대신, 일부씩 (`chunksize` 옵션 사용) 나누어 읽고 처리합니다. 이는 메모리 사용을 줄여 대용량 파일 처리에 효과적입니다.
2. **병렬 처리 (`multiprocessing`)**:
   - 병렬 처리를 사용하여 데이터를 분할하고, 파일 저장 작업을 동시에 여러 프로세스에서 처리합니다.
3. **진행 상태 시각화 및 실시간 업데이트**:
   - `tqdm`을 사용하여 파일이 어느 정도 진행되었는지 확인할 수 있도록 합니다.

아래 코드에서는 이러한 방법을 사용하여 대용량 Excel 파일을 좀 더 효율적으로 분할하고, ZIP으로 압축하는 작업을 수행합니다.

### 코드의 주요 변경 사항:
1. **`pandas`의 `chunksize` 사용**:
   - `pd.read_excel`을 사용할 때 `chunksize` 옵션을 추가하여 메모리에 부담이 가지 않도록 10,000행 단위로 데이터를 읽어옵니다.
   - `chunksize` 크기는 메모리 상태에 따라 조절할 수 있습니다. (필요에 따라 5,000 또는 20,000으로 조정)

2. **데이터를 리스트에 저장하여 결합**:
   - 데이터 청크를 리스트로 저장하여 나중에 필요할 때 빠르게 결합할 수 있도록 설정했습니다.
   
3. **`multiprocessing`을 통한 병렬 처리**:
   - `multiprocessing.Pool`을 사용하여 여러 파일을 동시에 생성하도록 하여 작업 시간을 단축했습니다.
   - `cpu_count()` 함수를 사용하여 시스템의 CPU 개수에 맞게 병렬 프로세스를 최대한 활용했습니다.
   
4. **진행률 표시**:
   - `tqdm` 라이브러리를 사용하여 파일 저장 및 압축 작업의 진행 상태를 시각적으로 표시했습니다.
   - `desc`와 `total` 매개변수를 사용하여 정확한 작업 상태를 표시했습니다.

### 코드 최적화 포인트:
- **메모리 절약**: `chunksize`를 사용하여 메모리 사용량을 줄이고, 병렬 처리를 통해 여러 작업을 동시에 수행하여 전체 실행 시간을 단축했습니다.
- **병렬 처리**: `multiprocessing`을 사용하여 분할 파일 저장 작업을 동시에 수행하여 속도를 개선했습니다.
- **진행 상태 표시**: `tqdm`을 사용하여 진행률을 쉽게 확인할 수 있도록 시각적인 피드백을 제공했습니다.

이 코드를 사용하면 대용량 Excel 파일을 좀 더 빠르고 효율적으로 분할하고 압축할 수 있을 것입니다. 추가적으로 필요한 사항이 있으면 언제든 알려주세요!


In [6]:
import os
import pandas as pd
import zipfile
from tqdm import tqdm
import time

# 타이머 시작
start_time = time.time()

# 원본 파일 경로 및 분할 파일 저장 경로
original_file = r'd:\시설\유지\24.9월유지가입자_보안해제2.xlsx'
split_dir = r'd:\시설\유지\분할파일'
zip_file_path = r'd:\시설\유지\유지가입자분할.zip'

# 분할할 파일 개수
split_count = 3

# 분할 파일 저장 폴더가 없으면 생성
if not os.path.exists(split_dir):
    os.makedirs(split_dir)

# 1. Excel 파일 읽기 타이머 측정
print("대용량 Excel 파일 읽는 중...")
read_start_time = time.time()
df = pd.read_excel(original_file, engine='openpyxl')
total_rows = len(df)
read_end_time = time.time()
print(f"파일 읽기 완료: {original_file} (총 {total_rows} 행)")
print(f"파일 읽기 소요 시간: {read_end_time - read_start_time:.2f} 초")

# 행 단위로 데이터를 나누기 위한 인덱스 설정
rows_per_split = total_rows // split_count

# 2. 분할 파일 생성 및 저장 타이머 측정
print("파일 분할 및 저장 중...")
split_start_time = time.time()
for i in tqdm(range(split_count), desc="분할 및 저장 진행", unit="파일"):
    start_row = i * rows_per_split
    end_row = (i + 1) * rows_per_split if i != split_count - 1 else total_rows
   
    # 데이터프레임 분할
    split_df = df.iloc[start_row:end_row]
   
    # 분할 파일 저장 경로 설정
    split_filename = os.path.join(split_dir, f'유지가입자_part_{i + 1}.xlsx')
   
    # 분할된 데이터프레임을 Excel 파일로 저장
    split_df.to_excel(split_filename, index=False)
    print(f"파일 생성 완료: {split_filename} (행: {start_row + 1} ~ {end_row})")
split_end_time = time.time()
print(f"파일 분할 및 저장 소요 시간: {split_end_time - split_start_time:.2f} 초")

# 3. ZIP 파일 압축 타이머 측정
print("파일 압축 작업 중...")
zip_start_time = time.time()
with zipfile.ZipFile(zip_file_path, 'w') as zipf:
    for root, _, files in os.walk(split_dir):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, arcname=file)  # 각 파일을 ZIP에 추가
            print(f"파일 압축 중: {file_path}")
zip_end_time = time.time()
print(f"파일 압축 소요 시간: {zip_end_time - zip_start_time:.2f} 초")

# 전체 소요 시간 출력
end_time = time.time()
total_elapsed_time = end_time - start_time
print(f"전체 작업 소요 시간: {total_elapsed_time:.2f} 초")


대용량 Excel 파일 읽는 중...


KeyboardInterrupt: 

In [7]:
import os
import pandas as pd
import zipfile
from tqdm import tqdm
import time

# 타이머 시작
start_time = time.time()

# 원본 파일 경로 및 분할 파일 저장 경로
original_file = r'd:\시설\유지\24.9월유지가입자_보안해제2.xlsx'
split_dir = r'd:\시설\유지\분할파일'
zip_file_path = r'd:\시설\유지\유지가입자분할.zip'

# 분할할 파일 개수
split_count = 3

# 분할 파일 저장 폴더가 없으면 생성
if not os.path.exists(split_dir):
    os.makedirs(split_dir)

# 1. Excel 파일 읽기
print("대용량 Excel 파일 읽는 중...")
df = pd.read_excel(original_file, engine='openpyxl')
total_rows = len(df)
print(f"파일 읽기 완료: {original_file} (총 {total_rows} 행)")

# 행 단위로 데이터를 나누기 위한 인덱스 설정
rows_per_split = total_rows // split_count

# 2. 분할 파일 생성 및 저장
print("파일 분할 및 저장 중...")
for i in tqdm(range(split_count), desc="분할 및 저장 진행", unit="파일"):
    start_row = i * rows_per_split
    end_row = (i + 1) * rows_per_split if i != split_count - 1 else total_rows
   
    # 데이터프레임 분할
    split_df = df.iloc[start_row:end_row]
   
    # 분할 파일 저장 경로 설정
    split_filename = os.path.join(split_dir, f'유지가입자_part_{i + 1}.xlsx')
   
    # 분할된 데이터프레임을 Excel 파일로 저장
    split_df.to_excel(split_filename, index=False)
    print(f"파일 생성 완료: {split_filename} (행: {start_row + 1} ~ {end_row})")

# 3. ZIP 파일 압축
print("파일 압축 작업 중...")
with zipfile.ZipFile(zip_file_path, 'w') as zipf:
    for root, _, files in os.walk(split_dir):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, arcname=file)  # 각 파일을 ZIP에 추가
            print(f"파일 압축 중: {file_path}")

# 전체 소요 시간 출력
end_time = time.time()
total_elapsed_time = end_time - start_time
print(f"전체 작업 소요 시간: {total_elapsed_time:.2f} 초")

대용량 Excel 파일 읽는 중...


KeyboardInterrupt: 

# 3개로 분리(용량해소)

In [1]:
import os
import zipfile
from openpyxl import load_workbook, Workbook

# 원본 파일 경로 설정 (.xlsx 파일로 확장자 변경)
original_file = r'd:\시설\유지\24.9월유지가입자_보안해제2.xlsx'

# 분할 파일 저장 경로 및 ZIP 파일 경로
split_dir = r'd:\시설\유지\분할파일'
zip_file_path = r'd:\시설\유지\유지가입자분할.zip'

# 분할할 파일 개수
split_count = 3

# 분할 파일 저장 경로가 없으면 생성
if not os.path.exists(split_dir):
    os.makedirs(split_dir)

# Excel 파일 읽기
wb = load_workbook(filename=original_file)
ws = wb.active

# 데이터 분할하기
rows = list(ws.iter_rows(values_only=True))  # 행 데이터를 리스트로 변환
total_rows = len(rows)  # 전체 행 개수
rows_per_split = total_rows // split_count  # 각 분할 파일당 행 개수

# 분할된 파일들을 생성 및 저장
for i in range(split_count):
    # 새로운 워크북 생성
    split_wb = Workbook()
    split_ws = split_wb.active
   
    # 각 분할 파일의 데이터 범위 설정
    start_row = i * rows_per_split
    end_row = (i + 1) * rows_per_split if i != split_count - 1 else total_rows

    # 해당 범위의 데이터를 새로운 워크북에 추가
    for row in rows[start_row:end_row]:
        split_ws.append(row)

    # 분할 파일 저장
    split_filename = os.path.join(split_dir, f'유지가입자_part_{i + 1}.xlsx')
    split_wb.save(split_filename)
    print(f"파일 생성 완료: {split_filename}")

# 분할된 파일을 ZIP 파일로 압축
with zipfile.ZipFile(zip_file_path, 'w') as zipf:
    for root, _, files in os.walk(split_dir):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, arcname=file)  # 각 파일을 ZIP에 추가
            print(f"파일 압축 중: {file_path}")

print(f"분할 파일이 성공적으로 ZIP 압축되었습니다: {zip_file_path}")

KeyboardInterrupt: 

## 주어진 Excel 파일을 처리하고 시각화할 수 있도록 예제 코드를 작성해보았습니다. 아래 코드는 `pandas`를 사용하여 Excel 파일의 시트를 선택하여 불러오고, 데이터 시각화를 위해 `pandas-profiling` 패키지를 사용한 예제입니다.

1. 파일명 변경:
   - `2024.09 마케팅부문 체납데이터(확정).xlsx` 파일의 확장자를 `.xlsx`로 변경하여 보안 해제.
   - 새로운 파일 이름은 `2024.09_체납_보안해제.xlsx`입니다.

2. 시트 선택 및 데이터 시각화:
   - `pandas`를 이용하여 여러 시트 중 하나를 선택하여 불러오고, 간단한 시각화와 데이터 분석 리포트를 생성하는 코드입니다.

In [ ]:
import pandas as pd
import os
from pandas_profiling import ProfileReport

# 파일 경로 설정
input_file_path = "D:/체납/10월체납활동/2024.09 마케팅부문 체납데이터(확정).xlsx"
output_file_path = "D:/체납/10월체납활동/2024.09_체납_보안해제.xlsx"

# 파일 확장자 변경
if os.path.exists(input_file_path):
    try:
        # 확장자 변경을 위해 파일 복사
        os.rename(input_file_path, output_file_path)
        print(f"파일명이 {output_file_path}로 변경되었습니다.")
    except Exception as e:
        print(f"파일명 변경 중 오류가 발생했습니다: {e}")
else:
    print("파일이 존재하지 않습니다. 경로를 확인하세요.")

# 엑셀 파일 읽기
try:
    # 엑셀 파일의 시트 목록 확인
    xls = pd.ExcelFile(output_file_path)
    print("사용 가능한 시트 목록:", xls.sheet_names)

    # 원하는 시트를 지정하여 데이터프레임으로 불러오기 (예: 첫 번째 시트)
    sheet_name = xls.sheet_names[0]  # 예: 첫 번째 시트
    df = pd.read_excel(output_file_path, sheet_name=sheet_name)

    # 불러온 데이터프레임 정보 출력
    print(f"\n'{sheet_name}' 시트에서 데이터를 성공적으로 불러왔습니다.")
    print(f"데이터프레임 형태: {df.shape}")
    print("컬럼 목록:", df.columns)

    # 데이터프레임 기본 통계 출력
    print("\n기본 통계 요약:")
    print(df.describe())

    # pandas-profiling 리포트 생성
    report = ProfileReport(df, title=f"Data Analysis Report - {sheet_name}", explorative=True)
    report_file_path = "D:/체납/10월체납활동/2024.09_체납_보안해제_Report.html"
    report.to_file(report_file_path)
    print(f"프로파일링 리포트가 {report_file_path}에 저장되었습니다.")

except Exception as e:
    print(f"데이터 처리 중 오류가 발생했습니다: {e}")

### 코드 설명:
1. **파일명 변경**: `os.rename()` 함수를 이용하여 기존 파일의 확장자를 변경하고, 보안 해제된 파일을 새로 생성합니다.
2. **시트 선택**: `pandas`의 `ExcelFile` 클래스를 사용하여 Excel 파일의 모든 시트를 확인하고, 첫 번째 시트를 선택하여 불러옵니다.
3. **데이터 분석**: `describe()` 메서드를 사용하여 기본 통계 요약을 확인하고, `pandas-profiling` 라이브러리로 데이터를 분석한 HTML 리포트를 생성합니다.
4. **리포트 저장**: 생성된 리포트는 `D:/체납/10월체납활동/` 폴더 내에 HTML 파일로 저장됩니다.

### 사전 설치해야 하는 라이브러리:
- `pandas`
- `pandas-profiling`

```bash
pip install pandas pandas-profiling
```

이 코드를 실행하여 원하는 데이터를 확인하고 시각화까지 진행할 수 있습니다. 데이터가 많을 경우, 시각화 작업의 속도가 느릴 수 있으니 필요에 따라 데이터를 필터링하거나 요약하여 사용하시기 바랍니다. 추가적으로 다른 시트를 선택하려면 `sheet_name` 값을 변경하면 됩니다.

In [ ]:
pip install pandas pandas-profiling